In [81]:
# import zipfile
# zip_ref=zipfile.ZipFile('book-recommendation-dataset.zip')
# zip_ref.extractall()
# zip_ref.close()

In [82]:
import pandas as pd
books=pd.read_csv('Books.csv')
users=pd.read_csv('Users.csv')
ratings=pd.read_csv('Ratings.csv')

C:\Users\alok4\AppData\Local\Temp\ipykernel_14108\3026388437.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv')


In [83]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [84]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [85]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [86]:
# Not Useful "Image UR-L", "Image UR-S", "Image UR-M", "Year of publication"
# The Image URLs will be useful for thumbnails on website

In [87]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [88]:
print(users.shape)
print(ratings.shape)
print(books.shape)

(278858, 3)
(1149780, 3)
(271360, 8)


In [89]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [90]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [91]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [92]:
books.duplicated().sum()

np.int64(0)

In [93]:
ratings.duplicated().sum()

np.int64(0)

In [94]:
users.duplicated().sum()

np.int64(0)

### Popularity Based Recommender System

Highest Rating Books, But we will conisder only those books that have atleast 250 ratings from users.

In [95]:
# Merge the ratings dataframe and the books dataframe on the ISBN Column

In [96]:
ratings_with_name=ratings.merge(books,on="ISBN")

In [97]:
ratings_with_name.shape # Ratings dataframe had some rows for which the ISBN was not there in the books dataframe

(1031136, 10)

In [98]:
df=ratings_with_name.groupby("Book-Title").mean(numeric_only=True)['Book-Rating'].reset_index()

In [99]:
df

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


Decide which columns we will keep. UserID won't be helpful, as we don't care about who did the rating.
Rather we will keep the  Book Title, Author, Publisher, 

In [100]:
num_rating_df=ratings_with_name.groupby("Book-Title").sum(numeric_only=True)['Book-Rating'].reset_index()
num_rating_df.rename(columns={"Book-Rating":'num_rating'},inplace=True)
num_rating_df

,Book-Title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,9
1,Always Have Popsicles,0
2,Apple Magic (The Collector's series),0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8
4,Beyond IBM: Leadership Marketing and Finance ...,0
...,...,...
241066,Ã?Â?lpiraten.,0
241067,Ã?Â?rger mit Produkt X. Roman.,21
241068,Ã?Â?sterlich leben.,7
241069,Ã?Â?stlich der Berge.,8


In [101]:
ratings_with_name[ratings_with_name['Book-Title']=="Ã?Â?rger mit Produkt X. Roman."]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
137083,35407,325721538X,6,Ã?Â?rger mit Produkt X. Roman.,Joan Aiken,1987,Diogenes Verlag,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...
271602,72601,325721538X,0,Ã?Â?rger mit Produkt X. Roman.,Joan Aiken,1987,Diogenes Verlag,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...
496838,133567,325721538X,8,Ã?Â?rger mit Produkt X. Roman.,Joan Aiken,1987,Diogenes Verlag,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...
829177,225343,325721538X,7,Ã?Â?rger mit Produkt X. Roman.,Joan Aiken,1987,Diogenes Verlag,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...,http://images.amazon.com/images/P/325721538X.0...


In [102]:
avg_rating_df=ratings_with_name.groupby("Book-Title").mean(numeric_only=True)['Book-Rating'].reset_index()
avg_rating_df.rename(columns={"Book-Rating":'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [103]:
pop_df=num_rating_df.merge(avg_rating_df,on='Book-Title')
pop_df

,Book-Title,num_rating,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,9,2.250000
1,Always Have Popsicles,0,0.000000
2,Apple Magic (The Collector's series),0,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,0,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,21,5.250000
241068,Ã?Â?sterlich leben.,7,7.000000
241069,Ã?Â?stlich der Berge.,8,2.666667


In [104]:
pop_df=pop_df[pop_df['num_rating']>=250].sort_values('avg_rating',ascending=False).head(50)
pop_df

,Book-Title,num_rating,avg_rating
70396,Free,449,8.017857
112160,Love You Forever,326,6.791667
64207,Falling Up,263,6.743590
208575,The Stand (The Complete and Uncut Edition),352,6.175439
77812,Griffin &amp; Sabine: An Extraordinary Corresp...,435,6.041667
80434,Harry Potter and the Prisoner of Azkaban (Book 3),2505,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),2254,5.824289
195672,The Little Prince,820,5.815603
180518,The Cat in the Hat,305,5.754717
80441,Harry Potter and the Sorcerer's Stone (Book 1),1595,5.737410


In [105]:
pop_df=pop_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Author','Book-Title','Image-URL-M','num_rating','avg_rating']]

In [106]:
pop_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/1844262553.01.MZZZZZZZ.jpg'

### Collaborative Filtering Based Recommneder System

In [107]:
x=ratings_with_name.groupby('User-ID').count()['Book-Rating']>200
padhe_likhe_users=x[x].index

In [108]:
filtered_rating=ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]
filtered_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1151,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,http://images.amazon.com/images/P/0026217457.0...,http://images.amazon.com/images/P/0026217457.0...
1152,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,http://images.amazon.com/images/P/003008685X.0...,http://images.amazon.com/images/P/003008685X.0...
1153,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,http://images.amazon.com/images/P/0030615321.0...,http://images.amazon.com/images/P/0030615321.0...
1154,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,http://images.amazon.com/images/P/0060002050.0...,http://images.amazon.com/images/P/0060002050.0...
...,...,...,...,...,...,...,...,...,...,...
1029357,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...
1029358,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...
1029359,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...
1029360,275970,4770019572,0,Musashi,Eiji Yoshikawa,1995,Kodansha International (JPN),http://images.amazon.com/images/P/4770019572.0...,http://images.amazon.com/images/P/4770019572.0...,http://images.amazon.com/images/P/4770019572.0...


In [114]:
# Finding the books with more than 50 times rating done

In [116]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index

In [118]:
final_ratings=filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [119]:
final_ratings.drop_duplicates()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1163,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
1165,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...
1168,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...
1174,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...
...,...,...,...,...,...,...,...,...,...,...
1029196,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...
1029197,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...
1029270,275970,1573229725,0,Fingersmith,Sarah Waters,2002,Riverhead Books,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...
1029309,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris,2001,Time Warner Audio Major,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...


In [123]:
pt=final_ratings.pivot_table(index='Book-Title',columns="User-ID",values="Book-Rating")

In [125]:
pt.fillna(0,inplace=True)

In [127]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(pt)

In [138]:
# Similarity score of each book with every other book

In [131]:
similarity_scores.shape

(706, 706)

In [132]:
import numpy as np

In [137]:
def recommend(book_name):
    index=np.where(pt.index==book_name)[0][0]
    
    similar_items=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    return similar_items

In [128]:
pt.shape

(706, 810)

In [109]:
import pickle
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))

NameError: name 'pt' is not defined